## Prueba NG Solve

Simulación

Primero se debe llamar a la gui (graphical user interphace), así esta se abre y luego cualquier simulación se observará en ella

In [1]:

# Importamos los paquetes
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *

def solve(h, draw = 0):
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))  # h será el tamaño de la malla y se usará el tipo unit_square, para otras geometrías hay que buscar el nombre y en base a ello se tendrán distintos boundries (se explicitan con strings ('bottom', 'top', etc))

    # Se definen funciones que se utilizarán, todas con lengauje simbólico.
    source = exp((x-0.5)*(y-0.5))*(2.5*x*y - y**4*x + y**3*x - 2*y**3*x - 2*y**3 - y**2*x**3 + y**2*x**2 - 0.5*y**2*x + y**2 - 3*y*x**2 - x)
    ud = exp((x-0.5)*(y-0.5))*x*y
    g = exp((x-0.5)*(y - 0.5))*(x*y + y**2*x - 0.5*y**2)


    fesp = H1(mesh, order=3, dirichlet="bottom")  # Se define un espacio de elementos finitos, H1 corresponde al espacio de polinomios continuos, se define también el tipo de condición según el borde (en este caso 'bottom')
    up, vp = fesp.TnT()                           # Se crean las funciones en el orden: trial function, test function

    #u = fes.TrialFunction()  # symbolic object   # Esta es una forma alternativa de generar la función de trial, test
    #v = fes.TestFunction()   # symbolic object
    #gfu = GridFunction(fes)  # solution

    ap = BilinearForm(fesp)                       # Se arma la forma bilineal (lado izquierdo de la formulación débil) (primero definiéndola para los elementos finitos diseñados)
    ap += (grad(up)*y*grad(vp) + up*vp)*dx        # Se coloca directamente la expresión
    ap.Assemble()                                 # Este paso es clave para armar la matriz que luego se usará en simular

    fp = LinearForm(fesp)                         # Se arma la forma lineal (lado derecho de la formulación débil)
    fp += source*vp*dx + g*vp * ds                # Se coloca directamente la expresión
    fp.Assemble()                                 # El ensamblaje del vector

    # con el comando 
    # ap.mat # se puede obtener la matriz
    # fp.vec # se puede obtener el vector

    gfup = GridFunction(fesp, "u-primal")         # Acá se guardará la solución
    gfup.Set(ud, BND)

    gfup.Set(g, definedon=mesh.Boundaries("bottom"))                   # Acá se agregar directamente las condiciones de borde

    r = fp.vec.CreateVector()                                            # Este vector permite que se puedan incorporar correctamente las condiciones de borde al sistema                
    r.data = fp.vec - ap.mat * gfup.vec                                  # Con esto    
    gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * fp.vec   # Con esto se resuelve el sistema
    if draw:
        Draw(gfup)
    return gfup, mesh
#Draw (grad(gfup), mesh, "flux-primal")

In [43]:
from ngsolve.krylovspace import CGSolver
import netgen.gui
#from netgen.geom2d import unit_square
from netgen.geom2d import SplineGeometry
from ngsolve import *

def initial_conds():
    v0 = x + sin(y)
    w0 = 0.2
    T = 20
    D = 1
    a = 1
    b = 1
    lamb = 1
    thet = 1
    params = (T, D, a, b, lamb, thet)
    return v0, w0, params

def solve_FHN1(v0, w0, params, I_app=0, h=0.2, order=2, CFL=0.7, draw=0, ion=False, DG=True, CG=False):
    """
    Usa euler explicito.
    """
    T, D, a, b, lamb, thet = params
    
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))  
    if DG:
        V = L2(mesh, order=order) # espacio de funciones integrables para 1ra ecuacion
        Q = L2(mesh, order=order-1) # espacio de funciones integrables para 2da ecuacion
    else:
        V = H1(mesh, order=order) # espacio de funciones con derivada integrable para 1ra ecuacion
        Q = H1(mesh, order=order-1) # espacio de funciones con derivada integrable para 2da ecuacion
    
    X = V*Q # espacio producto
    (v, p), (w, q) = X.TnT() # funciones trial y test


    a = BilinearForm(X, nonassemble=True)
    a += (D * grad(v) * grad(p)) * dx # primera ecuacion fitshugh-nagumo estatica (sin tiempo)
    if ion: 
        I_ion = (-lamb*v*(v - thet)*(1 - v)*p + lamb*w*p) * dx # termino no lineal (corriente ionica)
        a += I_ion
    a += (b*w*q)*dx # segunda ecuacion fitshugh-nagumo estatica # No funciona hacer v*q
    a.Assemble()

    if I_app:
        l = LinearForm(X)
        l += I_app * p * dx
        l.Assemble()

    m = X.Mass(1)
    if CG:
        prem = m.mat.CreateSmooter()
        minv = CGSolver(m, prem, tol=1e-8, maxiter=200)  # metodo para invertir más rápido la matriz (con gradiente conjugado)
    else:
        minv = m.Inverse()

    t = 0; dt = CFL*h

    gfu = GridFunction(X)
    gfu.components[0].Set(v0) # Se fija condicion inicial v
    gfu.components[1].Set(w0) # Se fija condicion inicial w

    #Draw(gfu.components[0], mesh,"u", autoscale=True, animate=False, order=5)

    res = gfu.vec.CreateVector()
    
    #Redraw(blocking=False)
    while t + dt/2 <= T:
        print(t)
        res.data = minv @ a.mat * gfu.vec
        gfu.vec.data -= dt * res
        t += dt
        #Redraw(blocking=False)
        #sceneu.Redraw()
    
    Draw(gfu.data, mesh, "a")
    return gfu

In [44]:
v0, w0, params = initial_conds()
gfu = solve_FHN1(v0, w0, params)


0
0.13999999999999999
0.27999999999999997
0.41999999999999993
0.5599999999999999
0.7
0.84
0.98
1.1199999999999999
1.2599999999999998
1.3999999999999997
1.5399999999999996
1.6799999999999995
1.8199999999999994
1.9599999999999993
2.099999999999999
2.2399999999999993
2.3799999999999994
2.5199999999999996
2.6599999999999997
2.8
2.94
3.08
3.22
3.3600000000000003
3.5000000000000004
3.6400000000000006
3.7800000000000007
3.920000000000001
4.0600000000000005
4.2
4.34
4.4799999999999995
4.619999999999999
4.759999999999999
4.899999999999999
5.039999999999998
5.179999999999998
5.319999999999998
5.459999999999997
5.599999999999997
5.739999999999997
5.879999999999996
6.019999999999996
6.159999999999996
6.299999999999995
6.439999999999995
6.579999999999995
6.719999999999994
6.859999999999994
6.999999999999994
7.1399999999999935
7.279999999999993
7.419999999999993
7.5599999999999925
7.699999999999992
7.839999999999992
7.9799999999999915
8.119999999999992
8.259999999999993
8.399999999999993
8.539999999

NgException: Archive error: Polymorphic type ngcomp::S_GridFunction<double> not registered for archive

In [12]:
Draw(gfu.components[0])

AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'Draw'

In [5]:
gfu.vec

basevector

In [3]:
def solve_FHN2(v0, w0, params, I_app=1, h=0.2, order=2, CFL=0.7, draw=0, ion=False, DG=False, CG=False):
    """
    Usa euler implicito. Por ahora la matriz del sistema queda no invertible
    """
    T, D, a, b, lamb, thet = params
    
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))  
    if DG:
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top") # espacio de funciones integrables para 1ra ecuacion
        Q = L2(mesh, order=order-1, dirichlet="bottom|right|left|top") # espacio de funciones integrables para 2da ecuacion
    else:
        V = H1(mesh, order=order, dirichlet="bottom|right|left|top") # espacio de funciones con derivada integrable para 1ra ecuacion
        Q = H1(mesh, order=order-1, dirichlet="bottom|right|left|top") # espacio de funciones con derivada integrable para 2da ecuacion
    
    X = V*Q # espacio producto
    (v, p), (w, q) = X.TnT() # funciones trial y test


    a = BilinearForm(X, symetric = False)  # le saqueé el nonasemble = False (no sé si le haga daño) y le agregué el symetric = False
    a += (D * grad(v) * grad(p)) * dx # primera ecuacion fitshugh-nagumo estatica (sin tiempo)
    if ion: 
        I_ion = (-lamb*v*(v - thet)*(1 - v)*p + lamb*w*p) * dx # termino no lineal (corriente ionica)
        a += I_ion
    a += (b*w*q)*dx # segunda ecuacion fitshugh-nagumo estatica # No funciona hacer v*q
    a.Assemble()

    if I_app:
        l = LinearForm(X)
        l += I_app * w * dx
        l.Assemble()

    m = BilinearForm(X, symetric = False)
    m += (v*p + q*w)*dx # Matriz de masa
    m.Assemble()
    #m = X.mass(1)

    t = 0; dt = CFL*h
    # ver parabolic.ipynb 
    if CG: # Usamos gradiente conjugado para invertir la matriz mas rapido
        mstar = m.mat + dt * a.mat
        premstar = m.mat.CreateSmoother() # + dt * a.mat.CreateSmoother()
        invmstar = CGSolver(mstar, premstar,\
                        tol=1e-8, maxiter=200)

    else:
        mstar = m.mat.CreateMatrix()
        #print(a.mat.AsVector() )
        mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
        invmstar = mstar.Inverse(freedofs=X.FreeDofs())

    gfu = GridFunction(X)
    gfu.components[0].Set(v0) # Se fija condicion inicial v
    gfu.components[1].Set(w0) # Se fija condicion inicial w

    sceneu = Draw(gfu,mesh,"u", autoscale=True, animate=False, order=5)

    res = gfu.vec.CreateVector()
    

    while t + dt/2 <= T:
        print(f'time {t}')
        res.data = dt * l.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        t += dt
        #sceneu.Redraw()
    Draw(gfu,mesh,"u", autoscale=True, animate=False, order=5)

In [4]:
v0, w0, params = initial_conds()
gfu = solve_FHN2(v0, w0, params)


time 0
time 0.13999999999999999
time 0.27999999999999997
time 0.41999999999999993
time 0.5599999999999999
time 0.7
time 0.84
time 0.98
time 1.1199999999999999
time 1.2599999999999998
time 1.3999999999999997
time 1.5399999999999996
time 1.6799999999999995
time 1.8199999999999994


None
